<a href="https://colab.research.google.com/github/TechWhizGenius/Teja_INFO5731_Fall2024/blob/main/TERM_PROJECT/Scrapping_Cornell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file contains scraping data from https://arxiv.org/ only

In [ ]:

# Installing necessary packages
!apt-get update
!apt-get install -y wget unzip
!pip install selenium


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,377 kB]
Get:14 https://de

In [ ]:
# Install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

# Check Chrome version
!google-chrome --version

--2024-10-28 23:05:47--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.16.136, 142.251.16.91, 142.251.16.190, ...
Connecting to dl.google.com (dl.google.com)|142.251.16.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111353576 (106M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 106.19M   116MB/s    in 0.9s    

2024-10-28 23:05:48 (116 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [111353576/111353576]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 123622 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (130.0.6723.69-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Package l

In [ ]:
# Downloading the correct version of ChromeDriver (matching Chrome 130.0.6723.58)

#!wget https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.58/linux64/chromedriver-linux64.zip

!wget -O chromedriver-linux64.zip https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.58/linux64/chromedriver-linux64.zip

# Unzip ChromeDriver directly to /usr/local/bin
!unzip chromedriver-linux64.zip -d /usr/local/bin/

# Move the extracted chromedriver to the correct location and set permissions
!sudo mv /usr/local/bin/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
!sudo chmod +x /usr/local/bin/chromedriver

# Verify ChromeDriver installation
!chromedriver --version


--2024-10-28 23:06:23--  https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.58/linux64/chromedriver-linux64.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.207, 64.233.180.207, 172.253.62.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9301462 (8.9M) [application/zip]
Saving to: ‘chromedriver-linux64.zip’

chromedriver-linux6 100%[===================>]   8.87M  --.-KB/s    in 0.06s   

2024-10-28 23:06:24 (153 MB/s) - ‘chromedriver-linux64.zip’ saved [9301462/9301462]

Archive:  chromedriver-linux64.zip
  inflating: /usr/local/bin/chromedriver-linux64/LICENSE.chromedriver  
  inflating: /usr/local/bin/chromedriver-linux64/THIRD_PARTY_NOTICES.chromedriver  
  inflating: /usr/local/bin/chromedriver-linux64/chromedriver  
ChromeDriver 130.0.6723.58 (3a50e012e4c9b8a410a4e2b12bf577e69ee8f755-refs/branch-heads/6723@{#1353})


In [ ]:
!google-chrome --version
!chromedriver --version


Google Chrome 130.0.6723.69 
ChromeDriver 130.0.6723.58 (3a50e012e4c9b8a410a4e2b12bf577e69ee8f755-refs/branch-heads/6723@{#1353})


Actual Code to Scrape data from https://arxiv.org/

In [ ]:
!pip install webdriver-manager


In [ ]:
# actual code

# Importing necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException

# Function to scrape papers
def scrape_arxiv(keyword, num_papers):
    # Reusing your initial Selenium setup
    options = Options()
    options.add_argument('--headless')  # Run headless Chrome
    options.add_argument('--no-sandbox')  # Bypass OS security model
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    options.add_argument('--disable-gpu')  # Disable GPU acceleration
    options.add_argument('--window-size=1920x1080')  # Set window size

    driver = webdriver.Chrome(service=Service('/usr/local/bin/chromedriver'), options=options)

    # List to store papers data
    papers_data = []

    # Base URL for arXiv search
    base_url = f'https://arxiv.org/search/?query={keyword}&searchtype=all&abstracts=show&order=-announced_date_first&size=50'

    # Loop through pages until reaching the desired number of papers
    for page in range(1, (num_papers // 50) + 2):  # 50 papers per page
        print(f'Scraping page {page}...')

        # Update the URL with the correct page number
        driver.get(base_url + f'&start={(page - 1) * 50}')
        time.sleep(2)  # Wait for the page to load

        # Wait until the papers are present on the page
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//li[@class="arxiv-result"]'))
        )

        # Find all paper elements on the current page
        papers = driver.find_elements(By.XPATH, '//li[@class="arxiv-result"]')

        # Loop through each paper and collect details
        for paper in papers:
            paper_info = {}

            try:
                # Extract title
                title_element = paper.find_element(By.XPATH, './/p[@class="title is-5 mathjax"]')
                paper_info['Title'] = title_element.text

                # Extract authors
                authors_element = paper.find_element(By.XPATH, './/p[@class="authors"]')
                paper_info['Authors'] = authors_element.text.replace('Authors:', '').strip()

                # Extract abstract
                abstract_element = paper.find_element(By.XPATH, './/p[@class="abstract mathjax"]')
                paper_info['Abstract'] = abstract_element.text

                # Extract submission date
                date_element = paper.find_element(By.XPATH, './/p[@class="is-size-7"]')
                submission_info = date_element.text.split(';')[0]  # Extracting submission date
                paper_info['Year'] = submission_info.split()[-1]  # Extracting year

                # Extract journal/conference with error handling
                try:
                    comments_element = paper.find_element(By.XPATH, './/p[@class="comments is-size-7"]')
                    if 'Accepted at' in comments_element.text:
                        paper_info['Journal/Conference'] = comments_element.text.split('Accepted at')[1].strip()
                    else:
                        paper_info['Journal/Conference'] = 'N/A'
                except NoSuchElementException:
                    paper_info['Journal/Conference'] = 'N/A'  # Default value if not found

                # Extract URL
                url_element = paper.find_element(By.XPATH, './/p[@class="list-title is-inline-block"]/a')
                paper_info['URL'] = url_element.get_attribute('href')

                # Generate DOI from the URL
                arxiv_id = url_element.text.split(':')[1].strip()
                paper_info['DOI'] = f'10.48550/arxiv.{arxiv_id}'

                # Append the paper info to the list
                papers_data.append(paper_info)

                # Stop if we reach the desired number of papers
                if len(papers_data) >= num_papers:
                    break

            except Exception as e:
                print(f"Error processing paper: {e}")
                continue

        if len(papers_data) >= num_papers:
            break

    driver.quit()  # Close the browser

    # Convert the list of papers to a DataFrame
    df = pd.DataFrame(papers_data)
    return df

# Usage example
keyword = '"Fleet Management for Autonomous Vehicles"'
num_papers = 2  # Limit to the desired number of papers
df_papers = scrape_arxiv(keyword, num_papers)

# Display the first few rows of the DataFrame
print(df_papers.head())

# Save to CSV
#df_papers.to_csv('arxiv_papers_autonomous_driving.csv', index=False)


Scraping page 1...
                                               Title  \
0  Fleet management for autonomous vehicles: Onli...   
1           Fleet management for autonomous vehicles   

                                             Authors  \
0  Sahar Bsaybes, Alain Quilliot, Annegret K. Wagler   
1  Sahar Bsaybes, Alain Quilliot, Annegret K. Wagler   

                                            Abstract  Year Journal/Conference  \
0  Abstract: The VIPAFLEET project consists in de...  2017                N/A   
1  Abstract: The VIPAFLEET project consists in de...  2016                N/A   

                                URL                        DOI  
0  https://arxiv.org/abs/1703.10565  10.48550/arxiv.1703.10565  
1  https://arxiv.org/abs/1609.01634  10.48550/arxiv.1609.01634  


In [ ]:
df_papers.shape

(2, 7)

In [ ]:
df_papers['Year'].unique()

array(['2017', '2016'], dtype=object)

In [ ]:
df_papers.head()

,Title,Authors,Abstract,Year,Journal/Conference,URL,DOI
0,Fleet management for autonomous vehicles: Onli...,"Sahar Bsaybes, Alain Quilliot, Annegret K. Wagler",Abstract: The VIPAFLEET project consists in de...,2017,N/A,https://arxiv.org/abs/1703.10565,10.48550/arxiv.1703.10565
1,Fleet management for autonomous vehicles,"Sahar Bsaybes, Alain Quilliot, Annegret K. Wagler",Abstract: The VIPAFLEET project consists in de...,2016,N/A,https://arxiv.org/abs/1609.01634,10.48550/arxiv.1609.01634


In [ ]:
df_papers.to_csv('Fleet_Management_for_Autonomous_Vehicles.csv', index=False)

from google.colab import files

files.download('Fleet_Management_for_Autonomous_Vehicles.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>